# Demonstration for the Molecule Dataset Preparation

by Martin Vögele

This demo requires Python 3 with pandas, numpy, pyplot, pytorch and RDKit

We start with some basic functions that we use to load and manipulate single molecules and to generate conformers. Then we introduce the central part of the workflow: the MoleculesDataset class. We show how to generate a dataset and to write it in various formats.

In [ ]:
# Standard modules
from __future__ import print_function, division
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# needed to make copies of data sets and variables
import copy

# Pytorch for data set
import torch
from torch.utils.data import Dataset, DataLoader

# RDkit 
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

# Molecule dataset definition
from molecules_dataset import *

## Basic Functions

### Reading and sorting data

We can read molecules from the SMILES provided in the respective dataset. 

In most cases, we add the missing hydrogens using RDKit. Hydrogens are added to satisfy the molecules valency for a total zero charge unless the charge of an atom is explicitly stated in the SMILES. We use all charges as stated in the SMILES of the respective dataset.

In [ ]:
test_mol = read_smiles('Nc1ccc(O)cc1')

In [ ]:
print(Chem.MolToMolBlock(test_mol))

By default, hydrogens are added to the molecule. But we can override this.

In [ ]:
test_mol_noh = read_smiles('Nc1ccc(O)cc1', add_h=False)

In [ ]:
print(Chem.MolToMolBlock(test_mol_noh))    

We can reorder the atoms such that hydrogens appear after the heavy atoms they are connected to.

In [ ]:
test_mol_ordered = reorder_atoms(test_mol)

In [ ]:
print(Chem.MolToMolBlock(test_mol_ordered))  

We can randomly reshuffle the atoms.

In [ ]:
test_mol_shuffled = reshuffle_atoms(test_mol)

In [ ]:
print(Chem.MolToMolBlock(test_mol_shuffled))    

### Generating conformers

We generate conformers of a molecule via ETKDG. This algorithm uses the distance geometry method [1], corrected with torsion angle preferences from Cambridge Structural Database (CSD) [2].
We then clean them up using RDKit's implementation of the Universal Force Field (UFF).

[1] Blaney, J. M.; Dixon, J. S., Reviews in Computational Chemistry, 1994.
[2] Riniker, S.; Landrum, G. A., J. Chem. Inf. Comp. Sci., 2015

In [ ]:
generate_conformers(test_mol,3)

In [ ]:
!mkdir examples
!mkdir examples/multiple_conformers

In [ ]:
for ic, conf in enumerate(test_mol.GetConformers()): 
    with open('examples/multiple_conformers/test'+str(ic)+'.xyz','w') as out_file:
        num_at = conf.GetNumAtoms()
        out_file.write('%i\ntest\n'%num_at)
        for ia in range(num_at):
            symbol   = test_mol.GetAtoms()[ia].GetSymbol()
            position = conf.GetAtomPosition(ia)
            xyz      = np.array([position.x, position.y, position.z])
            out_file.write("%s %8.5f %8.5f %8.5f\n"%(symbol, position.x, position.y, position.z))

In [ ]:
!cat examples/multiple_conformers/test*.xyz

We exclude all molecules for which no conformers were generated.

In [ ]:
from io import StringIO
import sys
Chem.WrapLogs()
sio = sys.stderr = StringIO()

error_mol = read_smiles('[Mg++].[O--].[Mg++].[Ca++]')
generate_conformers(error_mol,3)
  
if 'ERROR' in sio.getvalue():
    print(sio.getvalue()) 
 

Many algorithms learn on molecular graphs. The nodes of molecular graphs are the atoms and their connections are chemical bonds. Graphs can be represented by connectivity matrices.

In [ ]:
from scipy.sparse.csgraph import floyd_warshall

In [ ]:
cm = get_connectivity_matrix(test_mol_ordered)
print(cm)

In [ ]:
dm = floyd_warshall(cm)
print(dm)

In [ ]:
graph_dist = np.zeros([cm.shape[0],cm.shape[1],7])

In [ ]:
for i in range(7):
    graph_dist[:,:,i] = dm <= i+1

In [ ]:
graph_dist[0,14,:]

In [ ]:
bm = get_bonds_matrix(test_mol)

In [ ]:
bm

In [ ]:
np.unique(bm)

In [ ]:
bonds_one_hot = (np.array([0,1,1.5,2,3]) == bm[...,None]).astype(int)

In [ ]:
bonds_one_hot

We can also read conformers from existing SDF files.

## The datset class
The central part of this workflow is the molecules dataset class that handles all information about one dataset. One can add new features or writer functions later

In [ ]:
MoleculesDataset?

### Generating the dataset

In [ ]:
esol_logs = MoleculesDataset('datasets_raw/esol/delaney.csv',
                             ['ESOL predicted log solubility in mols per litre','measured log solubility in mols per litre'], 
                             num_conf=1, bond_order=True, max_num_at=66, id_name='Compound ID')

Each element (sample) of the dataset gives us all information we have on this molecule.

In [ ]:
esol_logs[1]

In [ ]:
blub = esol_logs[1]['data']
type(blub[0])

### Manipulating and exploring the dataset

We can copy datasets and modify them. Some modifying functions are already built in the class.

In [ ]:
esol_logs_mod = copy.deepcopy(esol_logs)

Here we add Gaussian noise to the molecular coordinates.
The width of the distribution is given in Angstroms.

In [ ]:
width=2

In [ ]:
esol_logs_mod.add_noise(width)

In [ ]:
differences = []
# For each molecule ...
for i,mol in enumerate(esol_logs_mod.coords):
    # ... for each conformer ...
    for j,conf in enumerate(mol):
        # ... and for each atom
        for k,atom in enumerate(conf):
            diff = esol_logs_mod.coords[i][j][k]-esol_logs.coords[i][j][k]
            differences.append(diff[0])
            differences.append(diff[1])
            differences.append(diff[2])

In [ ]:
plt.hist(differences,100)
plt.show()

Now let's get some statistics on how many molecules contain a certain element. 

We see, e.g., that all molecules contain carbon and almost all contain hydrogen.

In [ ]:
el_names,num_el_contained = esol_logs.element_statistics()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=[4,3],dpi=100)
ax.bar(el_names,num_el_contained)
ax.set_ylim(0,np.max(num_el_contained))
#plt.xticks(fontsize=12)
ax.set_xlim(-0.5,9.5)
ax.set_xlabel('element')
ax.set_ylabel('num. of mol. with this element')
fig.tight_layout()

### Splitting the dataset

We can also split the dataset in test, validation, and training sets (with manually definable random seed).

In [ ]:
test_indices, vali_indices, train_indices = esol_logs.split_randomly(random_seed=42)
print(test_indices)

With the function ```split_by_list```, we can provide lists of SMILES by which to split the dataset.

### Writing the dataset

In case we cannot directly feed the dataset into the code, we can write it in various formats.

There are three different writer functions. They have been designed to produce input data for Cormorant, for one of its predecessors, and for graph-based methods.

More writer functions can be added as necessary.

In [ ]:
esol_logs.write_connectivity_matrices('examples/logS_test.dat', prop_idx=1, indices=test_indices )
esol_logs.write_connectivity_matrices('examples/logS_vali.dat', prop_idx=1, indices=vali_indices )
esol_logs.write_connectivity_matrices('examples/logS_train.dat',prop_idx=1, indices=train_indices)

In [ ]:
esol_logs.write_compressed('examples/logS_test.npz', indices=test_indices, write_bonds=True)
esol_logs.write_compressed('examples/logS_vali.npz', indices=vali_indices, write_bonds=True)
esol_logs.write_compressed('examples/logS_train.npz',indices=train_indices,write_bonds=True)

In [ ]:
esol_logs.mol[0]

In [ ]:
logs_test = np.load('examples/logS_test.npz')
print(logs_test.files)
print(logs_test['bonds'].shape)
print(logs_test['bonds'][0])

In [ ]:
esol_logs.write_sdf_dataset('examples/logS_test', indices=test_indices)
esol_logs.write_sdf_dataset('examples/logS_vali', indices=vali_indices)
esol_logs.write_sdf_dataset('examples/logS_train',indices=train_indices)

In [ ]:
esol_logs.write_sdf_per_molecule('examples/logS_per_molecule_test', indices=test_indices)
esol_logs.write_sdf_per_molecule('examples/logS_per_molecule_vali', indices=vali_indices)
esol_logs.write_sdf_per_molecule('examples/logS_per_molecule_train',indices=train_indices)

In [ ]:
esol_logs.write_sdf_per_conformer('examples/logS_per_conformer_test', indices=test_indices)
esol_logs.write_sdf_per_conformer('examples/logS_per_conformer_vali', indices=vali_indices)
esol_logs.write_sdf_per_conformer('examples/logS_per_conformer_train',indices=train_indices)